In [1]:
import pandas as pd
import numpy as np
import datetime
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import auc
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter
from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from scipy.stats import entropy

In [2]:
from catboost import CatBoostClassifier

# 載入模型路徑
model_path = 'dataset_2nd/model.cbm'

# Create an instance of the CatBoostClassifier
loaded_model = CatBoostClassifier()

# Load the model from the file
loaded_model.load_model(model_path, format='cbm')


In [3]:
new_val_data = pd.read_parquet('val_data.parquet')
example = pd.read_csv('dataset_1st/31_範例繳交檔案.csv')
new_val_data = new_val_data.set_index('txkey')

In [4]:
columns_to_drop = ['chid', 'cano', 'bnsfg', 'flbmk', 'ovrlt', 'iterm']

cat_features = ['contp', 'etymd', 'mcc', 'ecfg', 'stocn', 'scity', 'insfg', 'mchno', 'acqic',
                'stscd', 'hcefg', 'csmcu', 'flg_3dsmk', 'hour','city_change', 'country_change']

X = new_val_data.drop(columns=columns_to_drop)

for feature in cat_features:
    X[feature] = X[feature].astype(str)
    
test_pool = Pool(X, cat_features=cat_features)


In [5]:
y_pred = loaded_model.predict(test_pool).astype(int)
new_val_data['pred']= y_pred
new_val_data =new_val_data.reset_index()

output_df = new_val_data[['txkey', 'pred']].set_index('txkey')
example = example.drop_duplicates(subset='txkey')

df2_sorted = example[['txkey']].merge(output_df, on='txkey', how='left')
df2_sorted = df2_sorted.set_index('txkey')

In [6]:
output_filename = 'dataset_2nd/predictions_secondround.csv'
df2_sorted.to_csv(output_filename, index='True')

In [7]:
df2_sorted.value_counts()

,pred
txkey,
a2c1209018e4e52e04f6fabb48f05f1b8bc09dc838ff6cb19906377fab414587,0
16c4880500059e01553789be11bbb50753b7acaae7b95bdcf6fb293a4d4a6e36,0
623c56be3bee87724e3d119c271d9ed098eeda842331830bcdcdee205329a531,0
250da12187059cf6e3a3066656a2919d08ceb8207efd550b3cf64eb47ae64d4d,0
4b268e0da036f44cbbb056ddfac6a28ea336d9cf299843bc7d40f3f7cb6c2d9f,0
...,...
47cd55bd7492e01e197b61799d072e6ca9e1c118bc76e924176566adab0d2b29,0
26cef9cc35fb73bedd08a711ebb6d612e4af7dd7b62e7c782aa58760bddd9cbc,0
f12f861ef6a8517ecddae262c1cc895f618e7e8c19eb9a768fc0b567e1e127d7,0


In [11]:
df2_sorted.value_counts()

pred
0       1348932
1          5389
Name: count, dtype: int64